In [5]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime

#CSV file adress
# adress = r"C:\Users\Cash\Documents\pruebas_python\proyectos\analisis de datos\archive (4)\calendar.csv"

#Parquet file adress
adress = r"calendar.parquet"

df = pd.read_parquet(adress)
df

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,167183,2021-04-15,f,$45.00,$45.00,1.0,5.0
1,6369,2021-04-15,t,$60.00,$60.00,1.0,1125.0
2,6369,2021-04-16,t,$60.00,$60.00,1.0,1125.0
3,6369,2021-04-17,t,$60.00,$60.00,1.0,1125.0
4,6369,2021-04-18,t,$60.00,$60.00,1.0,1125.0
...,...,...,...,...,...,...,...
7161251,49103093,2022-04-10,f,$28.00,$35.00,1.0,1125.0
7161252,49103093,2022-04-11,f,$28.00,$35.00,1.0,1125.0
7161253,49103093,2022-04-12,f,$28.00,$35.00,1.0,1125.0
7161254,49103093,2022-04-13,f,$28.00,$35.00,1.0,1125.0


In [3]:
#Turning adjusted_price to float(number value) after removing Nan values:
df["adjusted_price"].fillna("", inplace=True)
df["adjusted_price"] = df["adjusted_price"].str.replace("$","")
df["adjusted_price"] = df["adjusted_price"].str.replace(",","")
df["adjusted_price"] = pd.to_numeric(df["adjusted_price"])
#----------------------------------------------------------------
df["price"].fillna("", inplace=True)
df["price"] = df["price"].str.replace("$","")
df["price"] = df["price"].str.replace(",","")
df["price"] = pd.to_numeric(df["price"])

In [5]:
#Comparison between total rents in april 2021 and april 2022(make sure to get the name of the host)

rents_2021 = df.loc[(df["date"].dt.year == 2021) & (df["date"].dt.month == 4)].groupby("listing_id")["date"].count().reset_index()
rents_2022 = df.loc[(df["date"].dt.year == 2022) & (df["date"].dt.month == 4)].groupby("listing_id")["date"].count().reset_index()

total_rents = rents_2021.merge(rents_2022, how = "left", left_on = "listing_id", right_on = "listing_id")

values = {"date_x" : "Total rents 2021","date_y" : "Total rents 2022"}
total_rents.rename(columns = values, inplace = True)

total_rents

# listing= r"C:\Users\Cash\Documents\pruebas_python\proyectos\analisis de datos\archive (4)\listings.csv"
listing = r"listings.csv"

df_listing = pd.read_csv(listing, low_memory = False)
df_listing

total_rents = total_rents.merge(df_listing, how = "left", left_on = "listing_id", right_on = "id")

values = total_rents.columns.to_list()
not_delete = ['listing_id','Total rents 2021','Total rents 2022','name','host_name']
for value in values:
    if value not in not_delete:
        total_rents.drop(value,axis = 1, inplace = True)
    

columns_insert = total_rents.pop("host_name")
total_rents.insert(1,"host_name", columns_insert)
total_rents
        

,listing_id,host_name,Total rents 2021,Total rents 2022,name
0,6369,Simon,16,14,"Rooftop terrace room , ensuite bathroom"
1,21853,Abdel,15,15,Bright and airy room
2,23001,Jesus,16,15,Apartmento Arganzuela- Madrid Rio
3,24805,A,16,14,Gran Via Studio Madrid
4,26825,Agustina,16,14,Single Room whith private Bathroom
...,...,...,...,...,...
19612,49185822,Belaid,16,14,Habitación con TV con Netflix en Lavapiés
19613,49186179,Belaid,15,15,Habitación con TV con Netflix en Gaztambide
19614,49187258,Belaid,16,14,Habitación con TV con Netflix en Chamberí
19615,49187471,Belaid,16,14,Habitación con TV con Netflix en Goya


In [6]:
#Get the 5 most expensive flats in 2021 with more avaibility than 180 days
dfx = df.merge(df_listing, how = "left", left_on = "listing_id", right_on = "id")

not_delete = ['listing_id','date','available','price_x','adjusted_price','name','host_name','availability_365']
values = dfx.columns.to_list()
for value in values:
    if value not in not_delete:
        dfx.drop(value, axis = 1, inplace = True)
        
        
dfx.loc[dfx["availability_365"]>180].sort_values(by= "price_x", ascending = False).head(5)

,listing_id,date,available,price_x,adjusted_price,name,host_name,availability_365
5257515,40483113,2022-01-17,f,10499.0,9449.0,Hab. 2 personas H.Victoria 4 -Puerta del Sol,Hotel Victoria 4,343
5257501,40483113,2022-01-03,t,10499.0,9449.0,Hab. 2 personas H.Victoria 4 -Puerta del Sol,Hotel Victoria 4,343
5257500,40483113,2022-01-02,t,10499.0,9449.0,Hab. 2 personas H.Victoria 4 -Puerta del Sol,Hotel Victoria 4,343
5257524,40483113,2022-01-26,f,10499.0,9449.0,Hab. 2 personas H.Victoria 4 -Puerta del Sol,Hotel Victoria 4,343
5257523,40483113,2022-01-25,f,10499.0,9449.0,Hab. 2 personas H.Victoria 4 -Puerta del Sol,Hotel Victoria 4,343


In [180]:
#Get the 5 most expensive flats per neighbouhood_group
df1 = df_listing.copy()
dfx = df.merge(df_listing, how = "left", left_on = "listing_id", right_on = "id")
not_delete = ['listing_id','date','price_x','name','host_name','neighbourhood_group']
hoods = list(dfx["neighbourhood_group"].unique())
values = dfx.columns.to_list()
for value in values:
    if value not in not_delete:
        dfx.drop(value, axis = 1, inplace = True)

index_value = []
for hood in hoods:
    index_value.extend(list(dfx.loc[dfx["neighbourhood_group"] == hood ].nlargest(5, "price_x").index))
dfx.loc[dfx["neighbourhood_group"] == "Hortaleza"].nlargest(5, "price_x").index


dfx.loc[index_value]


,listing_id,date,price_x,name,host_name,neighbourhood_group
2246674,21119642,2022-01-01,9999.0,Ever House IFEMA - Mar de Cristal,Miguel Angel,Hortaleza
2246675,21119642,2022-01-02,9999.0,Ever House IFEMA - Mar de Cristal,Miguel Angel,Hortaleza
2246676,21119642,2022-01-03,9999.0,Ever House IFEMA - Mar de Cristal,Miguel Angel,Hortaleza
2246677,21119642,2022-01-04,9999.0,Ever House IFEMA - Mar de Cristal,Miguel Angel,Hortaleza
2246678,21119642,2022-01-05,9999.0,Ever House IFEMA - Mar de Cristal,Miguel Angel,Hortaleza
...,...,...,...,...,...,...
6235341,44245065,2021-09-09,9999.0,DOBO HOMES CALIZA APARTMENT,Dobo Rooms,Vicálvaro
6235342,44245065,2021-09-10,9999.0,DOBO HOMES CALIZA APARTMENT,Dobo Rooms,Vicálvaro
6235343,44245065,2021-09-11,9999.0,DOBO HOMES CALIZA APARTMENT,Dobo Rooms,Vicálvaro
6235344,44245065,2021-09-12,9999.0,DOBO HOMES CALIZA APARTMENT,Dobo Rooms,Vicálvaro


In [182]:
#Get all the hotel rooms that are in Salamanca and have more than 50 reviews
df_listing.loc[
    
    (df_listing["neighbourhood_group"] == "Salamanca") &
    (df_listing["room_type"].str.contains("Hotel")) &
    (df_listing["number_of_reviews"] > 50)

]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2631,11883513,Great studio in Salamanca neighborhood,51564174,Álvaro & Tatiana,Salamanca,Castellana,40.43343,-3.67959,Hotel room,87,1,57,2021-03-10,0.99,24,234
3743,15812097,Apartamento con piscina -Barrio Salamanca-,39011655,Adolfo,Salamanca,Lista,40.43110,-3.67373,Hotel room,55,3,111,2020-07-24,2.07,1,196
6018,21274204,Bright apartment in Goya perfect for groups!,51564174,Álvaro & Tatiana,Salamanca,Goya,40.42421,-3.67093,Hotel room,271,2,53,2020-08-28,1.30,24,0
